## **Catégorisez automatiquement des questions**

### partie 2/8 : requête API

#### Un notebook de requête API, qui récupère 50 questions par requête via le wrapper StackAPI ou bien via Stack Exchange API, et qui stocke les principales caractéristiques de ces questions dans un DataFrame.

<br>


## 1 Imports


In [22]:
import requests
import datetime
import pandas as pd


## 2 Requete API, via Stack Exchange API : objectif


In [23]:
# "réaliser une requête de 50 questions sur une période définie, contenant le tag “python”
# et qui ont un score > 50 (votes), récupérer les données principales de la question
# (date, titre, tags, score) dans un DataFrame et les afficher"


## 3 Authentification, RGPD (filtre)


In [24]:
# Authentification
# Pas nécessaire ici, nous avons droit à 10 000 requêtes(lecture)/j sans OAuth 2.0
# L'OAuth 2.0 est indispensable seulement pour les requetes en écriture


# RGPD
# Nous n'avons aucune raison d'extraire des données personnelles ici, donc nous éviterons non seulement
# certaines colonnes évidentes, par exemple :
# OwnerUserId, OwnerDisplayName, LastEditorUserId, LastEditorDisplayName, etc...
# mais aussi la clé primaire (l'id), qui permettrait de les retrouver
# (notre cle primaire, si nous en avons besoin, sera simplement l'index de notre dataframe)

# Remarque /RGPD :
# github offre une sécurité supplémentaire avec github secrets,
# qui nous informe si des failles de sécurité sont détectées
# (même si le fichier est compressé !)

# Nous allons donc commencer par créer un filtre.
# On peut le faire depuis https://api.stackexchange.com/docs/create-filter,
# Ou directement depuis ce notebook, en utilisant par exemple le module requests.

api_filter_url = 'https://api.stackexchange.com/2.3/filters/create'
filter_params = {
    'include': 'creation_date,title,tags,score',
    'unsafe': 'false',
}

response = requests.get(api_filter_url, params=filter_params)
data = response.json()
print(data)


{'items': [{'included_fields': ['.backoff', '.error_id', '.error_message', '.error_name', '.has_more', '.items', '.quota_max', '.quota_remaining', 'access_token.access_token', 'access_token.account_id', 'access_token.expires_on_date', 'access_token.scope', 'account_merge.merge_date', 'account_merge.new_account_id', 'account_merge.old_account_id', 'achievement.account_id', 'achievement.achievement_type', 'achievement.badge_rank', 'achievement.creation_date', 'achievement.is_unread', 'achievement.link', 'achievement.on_site', 'achievement.reputation_change', 'achievement.title', 'answer.answer_id', 'answer.collectives', 'answer.community_owned_date', 'answer.content_license', 'answer.creation_date', 'answer.is_accepted', 'answer.last_activity_date', 'answer.last_edit_date', 'answer.locked_date', 'answer.owner', 'answer.posted_by_collectives', 'answer.question_id', 'answer.recommendations', 'answer.score', 'article.article_id', 'article.article_type', 'article.creation_date', 'article.las

In [25]:
# ??

api_filter_url = 'https://api.stackexchange.com/2.3/filters/create'
filter_params = {
    'include': 'creation_date,title,tags,score',
    'unsafe': 'false',
}

response = requests.post(api_filter_url, data=filter_params)
data = response.json()
print(data)


{'items': [{'included_fields': ['.backoff', '.error_id', '.error_message', '.error_name', '.has_more', '.items', '.quota_max', '.quota_remaining', 'access_token.access_token', 'access_token.account_id', 'access_token.expires_on_date', 'access_token.scope', 'account_merge.merge_date', 'account_merge.new_account_id', 'account_merge.old_account_id', 'achievement.account_id', 'achievement.achievement_type', 'achievement.badge_rank', 'achievement.creation_date', 'achievement.is_unread', 'achievement.link', 'achievement.on_site', 'achievement.reputation_change', 'achievement.title', 'answer.answer_id', 'answer.collectives', 'answer.community_owned_date', 'answer.content_license', 'answer.creation_date', 'answer.is_accepted', 'answer.last_activity_date', 'answer.last_edit_date', 'answer.locked_date', 'answer.owner', 'answer.posted_by_collectives', 'answer.question_id', 'answer.recommendations', 'answer.score', 'article.article_id', 'article.article_type', 'article.creation_date', 'article.las

In [26]:
# La stack exchange API inclue des champs par defaut, la doc conseille de filtrer
# manuellement la réponse.


## 4 Stratégies possibles


In [27]:
# On peut utiliser directement l'outil en ligne, qui fournit une reponse de 100 questions max,
# format JSON compressé.
# exemple de la doc :
# /2.3/questions?fromdate=1320969600&todate=1321660800&order=desc&min=10&sort=votes&tagged=skyrim&site=gaming

# Avant de récupérer nos questions, nous pouvons d'abord nous faire une idée du nombre de questions
# qui satisfont nos critères sur une période donnée.

# Pour cela il y a plusieurs stratégies possibles,
# on peut par exemple utiliser le filtre "total" (built-in)
# Ou la méthode/endpoint "search" (plus flexible)
# Comme nous allons au final récupérer des questions, utilisons directement la méthode questions


## 5 Test : nb de questions valides cette année


In [28]:
# All dates in the API are in unix epoch time

def get_number_of_questions(tag, start_date, end_date, min_score=50, pagesize=50):
    """
    Get the number of questions from Stack Exchange API based on specified criteria.

    Parameters:
    - tag (str): The tag to filter questions.
    - start_date (datetime): The start date for the query.
    - end_date (datetime): The end date for the query.
    - min_score (int): Minimum score for the questions (default: 50).
    - page_size (int): Number of questions per page (default API: 30, max: 100).

    Returns:
    - int: Number of questions matching the criteria.
    """
    api_url = 'https://api.stackexchange.com/2.3/questions'

    params = {
        'order': 'desc',
        'sort': 'votes',
        'tagged': tag,
        'site': 'stackoverflow',
        'fromdate': int(start_date.timestamp()),
        'todate': int(end_date.timestamp()),
        'min': min_score,
        'pagesize': pagesize
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()

        if 'items' in data:
            return len(data['items'])
        else:
            return 0

    except requests.RequestException as e:
        print(f"Error during API request: {e}")
        return 0

# Observons l'activité cette année
tag='python'
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 11, 27)

result = get_number_of_questions(tag, start_date, end_date, pagesize=100)

print(f'Number of questions with the tag "{tag}" between {start_date} and {end_date}, votes > 50: \n{result}')


Number of questions with the tag "python" between 2023-01-01 00:00:00 and 2023-11-27 00:00:00, votes > 50: 
13


## 6 nb de questions valides l'année derniere


In [29]:
# Seulement 10 questions crées avec le tag python et un score > 50 (en presque un an)

# Observons l'activité l'année dernière
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

result = get_number_of_questions(tag, start_date, end_date)
print(f'Number of questions with the tag "{tag}" between {start_date} and {end_date}, votes > 50: \n{result}')

# Il faut du temps pour que le score atteigne 50, mais le traffic sur stack exchange a
# peut-être aussi diminué depuis l'année dernière (arrivée de chatGPT, etc...) ?


Number of questions with the tag "python" between 2022-01-01 00:00:00 and 2022-12-31 00:00:00, votes > 50: 
37


In [30]:
# Juste pour savoir

# Observons l'activité cette année, sur 1 semaine
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 1, 8)

result = get_number_of_questions(tag, start_date, end_date, min_score=5)
print(f'Number of questions 2023: \n{result}')

# Puis l'année dernière
start_date_2022 = datetime.datetime(2022, 1, 1)
end_date_2022 = datetime.datetime(2022, 1, 8)

result2 = get_number_of_questions(tag, start_date_2022, end_date_2022, min_score=5)
print(f'Number of questions 2022: \n{result2}')

# Il semble que c'est bien le score qui limite le nombre de réponses.


Number of questions 2023: 
15
Number of questions 2022: 
50


## 7 Notre requete


In [31]:
def get_questions(tag, start_date, end_date, min_score=50, pagesize=50,
                  api_url='https://api.stackexchange.com/2.3/questions',
                  site = 'stackoverflow'):
    """
    Get questions from Stack Exchange API based on specified criteria.

    Parameters:
    - tag (str): The tag to filter questions.
    - start_date (datetime): The start date for the query.
    - end_date (datetime): The end date for the query.
    - min_score (int): Minimum score for the questions (default: 50).
    - page_size (int): Number of questions per page (default API: 30, max: 100. We want 50).

    Returns:
    - questions formatted as JSON object
    """
    api_url = api_url

    params = {
        'order': 'desc',
        'sort': 'votes',
        'tagged': tag,
        'site': site,
        'fromdate': int(start_date.timestamp()),
        'todate': int(end_date.timestamp()),
        'min': min_score,
        'pagesize': pagesize  # 30 par défaut, 100 max, 50 ici
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()
        return data

    except requests.RequestException as e:
        print(f"Error during API request: {e}")
        return 0


# Prenons une période récente, et suffisante
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 11, 27)

result = get_questions(tag, start_date, end_date)
display(result)


{'items': [{'tags': ['python', 'pip', 'packaging', 'pyproject.toml'],
   'owner': {'account_id': 3400218,
    'reputation': 29683,
    'user_id': 3015186,
    'user_type': 'registered',
    'accept_rate': 50,
    'profile_image': 'https://i.stack.imgur.com/KagUg.jpg?s=256&g=1',
    'display_name': 'Niko Fohr',
    'link': 'https://stackoverflow.com/users/3015186/niko-fohr'},
   'is_answered': True,
   'view_count': 284062,
   'accepted_answer_id': 66472800,
   'answer_count': 5,
   'score': 351,
   'last_activity_date': 1677001110,
   'creation_date': 1595181079,
   'last_edit_date': 1614931841,
   'question_id': 62983756,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/62983756/what-is-pyproject-toml-file-for',
   'title': 'What is pyproject.toml file for?'},
  {'tags': ['python', 'pandas', 'xlrd', 'pcf'],
   'owner': {'account_id': 20190357,
    'reputation': 2909,
    'user_id': 14808721,
    'user_type': 'registered',
    'profile_image': 'http

## 8 Filtrage et affichage : le dataframe obtenu


In [32]:
# Plus qu'à filtrer / créer le dataframe

def extract_fields_and_create_dataframe(data):
    questions = data.get('items', [])

    # Extracting desired fields
    extracted_data = [
        {
            'creation_date': question.get('creation_date', None),
            'title': question.get('title', None),
            'tags': question.get('tags', None),
            'score': question.get('score', None),
        }
        for question in questions
    ]

    # Creating DataFrame
    df = pd.DataFrame(extracted_data)

    return df


df = extract_fields_and_create_dataframe(result)

print(df.shape)
display(df)


(50, 4)


,creation_date,title,tags,score
0,1595181079,What is pyproject.toml file for?,"[python, pip, packaging, pyproject.toml]",351
1,1607702016,xlrd.biffh.XLRDError: Excel xlsx file; not sup...,"[python, pandas, xlrd, pcf]",289
2,1580572695,What does model.eval() do in pytorch?,"[python, machine-learning, deep-learning, pyto...",264
3,1606035195,docker.errors.DockerException: Error while fet...,"[python, linux, docker, docker-compose]",227
4,1612516308,"ValueError: numpy.ndarray size changed, may in...","[python, pandas, numpy, scikit-learn, python-3.7]",227
5,1593658606,sqlalchemy.exc.NoSuchModuleError: Can&#39;t lo...,"[python, postgresql, sqlalchemy, flask-sqlalch...",226
6,1608242137,Python was not found; run without arguments to...,"[python, python-3.x, windows-10]",208
7,1677700339,How do I solve &quot;error: externally-managed...,"[python, error-handling, pip]",206
8,1648058522,How can I fix the &quot;zsh: command not found...,"[python, macos, terminal, atom-editor, macos-m...",205
9,1604676159,DeprecationWarning: executable_path has been d...,"[python, selenium]",201


## 9 Requete sur le site fr


In [33]:
# test fr, sans tag

def get_questions_fr(start_date, end_date, min_score=50, pagesize=50,
                  api_url='https://api.stackexchange.com/2.3/questions',
                  site = 'french'):
    """
    Get questions from Stack Exchange API based on specified criteria.

    Parameters:
    - start_date (datetime): The start date for the query.
    - end_date (datetime): The end date for the query.
    - min_score (int): Minimum score for the questions (default: 50).
    - page_size (int): Number of questions per page (default API: 30, max: 100. We want 50).

    Returns:
    - questions formatted as JSON object
    """
    api_url = api_url

    params = {
        'order': 'desc',
        'sort': 'votes',
        'site': site,
        'fromdate': int(start_date.timestamp()),
        'todate': int(end_date.timestamp()),
        'min': min_score,
        'pagesize': pagesize  # 30 par défaut, 100 max, 50 ici
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()
        return data

    except requests.RequestException as e:
        print(f"Error during API request: {e}")
        return 0


#
start_date = datetime.datetime(2010, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

result_fr = get_questions_fr(start_date=start_date, end_date=end_date,
                       min_score=0, pagesize=50, site='french')

df_fr = extract_fields_and_create_dataframe(result_fr)

print(df_fr.shape)
display(df_fr)

# interesting :)
# par contre le Stack Exchange Data Explorer (SEDE) ne permet pas de filtrer par site,
# donc impossible de recuperer 50 000 questions d'un coup.

# autres strats possibles si on veut entrainer un modele fr :
# - apres authentification, on pourrait le faire par requetes automatiques avec l'api, en 2 jours environ
# (nb requetes limite 300 par j + limite / minute. il faut y aller doucement)
# - trouver une autre sources de questions + tags en fr


(50, 4)


,creation_date,title,tags,score
0,1313616378,Pourquoi place-t-on une espace avant les ponct...,"[histoire, typographie, ponctuation]",105
1,1313611862,If &quot;Je t&#39;aime&quot; means &quot;I lov...,[usage],87
2,1313665489,Pourquoi trente-six&#160;?,"[expressions, nombres, origine-ou-raison]",83
3,1313618007,Accentuation des majuscules — Accents on upper...,"[orthographe, typographie, diacritiques, majus...",73
4,1313740740,When does one pronounce the &#39;s&#39; in plus?,"[prononciation, n&#233;gation, comparatifs]",73
5,1314356872,How to translate “By the way”? — Comment tradu...,"[traduction, anglais, expressions, oral]",59
6,1487234147,What languages are perceived as classy or fanc...,"[histoire, comparaison-de-langues, emprunts, r...",59
7,1531154732,Where did French&#39;s silent ending consonant...,"[prononciation, &#233;tymologie, histoire, ori...",51
8,1313819407,Quand peut-on mettre un adjectif avant ou apr&...,"[grammaire, adjectifs, ordre-des-mots]",49
9,1358529089,"Why are O and E sometimes attached together, a...","[orthographe, alphabet, ligatures]",47


## Conclusion


In [34]:
# Mon retour sur l'utilisation de l'API :

# La doc est très claire, elle va à l'essentiel dès le début et elle est courte et très bien structurée :
# Il y a très vite la liste des méthodes/endpoints, et des exemples de requetes un peu complexes
# pour comprendre et tester la syntaxe.

# Je ne l'ai pas assez manipulée pour pouvoir vraiment proposer des améliorations.
# Peut-être créer un filtre spécifique aux RGPD, et simplifier l'accès aux filtres publics ?


## Annexes : StackExchange Data Explorer


In [35]:
# La page d'accueil du "StackExchange Data Explorer" (l'outil d’export de données)
# présente la structure de la table, avec le type de chaque colonne, et un lien vers la doc :
# https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

# Table : posts
# "You find in Posts all non-deleted posts."

# La requete donnee en exemple est un tres bon pt de depart.
# Tags1 - question tags (PostTypeId = 1)
# AnswerCount - the number of undeleted answers (only present if PostTypeId = 1)
# FavoriteCount (nullable)

# On peut aussi y ajouter :

# extraire des questions relativement recentes : par exemple, parce que de nouveaux tags on pu apparaitre,
# qui seraient sous-estimes autrement.
# CreationDate > ...


# autres pistes possibles, mais pas tres utiles je pense :

# CommentCount (nullable) : semble redondant puisqu'on a deja AnswerCount et FavoriteCount
# de + sur stack il y a generalemt bcp - de commentaires que de reponses
# ClosedDate1 (present only if the post is closed) :  pas sur que cela nous importe directement
# CommunityOwnedDate (present only if post is community wiki'd) donc pas pour les questions
# ContentLicense1 pas pertinent pour nous.


## Requete SQL


In [36]:
# la requete proposee en exemple

# SELECT TOP 50000 Title, Body, Tags, Id, Score, ViewCount, FavoriteCount, AnswerCount
# FROM Posts
# WHERE PostTypeId = 1 AND ViewCount > 10 AND FavoriteCount > 10
# AND Score > 5 AND AnswerCount > 0 AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5

# donne 1 seule reponse ! Baissons le favoriteCount

# SELECT TOP 50000 Title, Body, Tags, Id, Score, ViewCount, FavoriteCount, AnswerCount
# FROM Posts
# WHERE PostTypeId = 1 AND ViewCount > 10 AND FavoriteCount > 5
# AND Score > 5 AND AnswerCount > 0 AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5

# Ok, seules 6 lignes ont un FavoriteCount > 1
# (136 lignes > 0)

# SELECT TOP 50000 Title, Body, Tags, Score, ViewCount, FavoriteCount, AnswerCount
# FROM Posts
# WHERE PostTypeId = 1 AND ViewCount > 10
# AND Score > 5 AND AnswerCount > 0 AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5

# fournit bien 50 000 lignes en reponse.

# est-ce qu on a vraimt besoin d'extraire les counts ?
# pourquoi pas juste Title, Body, Tags ?
# Pas besoin de GROUP BY ou d'ORDER BY ici.

# On peut choisir d'extraire, parmi nos questions qui remplissent nos criteres,
# les plus recentes. Notre requete devient :

# SELECT TOP 50000 Title, Body, Tags, Id, Score, ViewCount, FavoriteCount, AnswerCount, CreationDate
# FROM Posts
# WHERE PostTypeId = 1 AND ViewCount > 10000 AND Score > 10 AND AnswerCount > 1
# AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5
# ORDER BY CreationDate DESC;
